In [ ]:
import re
import pandas as pd
import numpy as np
# Read HTML module
from pyquery import PyQuery as pq

# Set the jupyter to display all the columns adn rows
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 800)

In [ ]:
# download dataset from MTA webset

def download_data(url,start,stop,log_name):
    """
    This function download selected datasets(prior 10/18/14) from MTA website, by finding the selected datasets by HTML elements.
    Then consolidate all the downloaded datasets to one large dataframe.
    Also, log the downloaded files information, such as file date, file_link, number of rows, and number of columns.
    """
    
    col_name_new = ['C/A','UNIT','SCP','STATION','LINENAME','DIVISION','DATE','TIME','DESC','ENTRIES','EXITS']
    df = pd.DataFrame(columns = col_name_new)
    

    # Log of downloading
    logFile  = open(log_name, 'w')  # 'Download_Log.txt'
    logFile.write('file_date;file_link;number_of_rows;number_of_columns')
    
    # get HTML code
    jpy = pq(url) 

    for i in range(start, stop, 2):
        # retrieve file path
        item = jpy('#contentbox > div > div > a:nth-child({})'.format(i))
        fileDate = item.text()
        filePath = 'http://web.mta.info/developers/' + item.attr('href')

        # get data from txt, save to csv, and append to datafram
        data = pd.read_csv(filePath, sep=",", header=0, names = col_name_new)
        df = df.append(data, ignore_index=True)
        # data.to_csv('./raw_data/MTA_data_{}.csv'.format(re.sub(r',','',fileDate)))

        # log the downloaded file information
        record = str(fileDate) + ';' + str(filePath) + ';' + str(data.shape[0]) + ';' + str(data.shape[1])
        logFile.write('\n' + record)
    
    logFile.close()
    
    return df

def remove_irregular_event(df):
    """
    In the original data, there are factors that may impact the data. (ie. Hardware failure, "IRREGULAR" audit event)
    Clean the data to filter out 'IRREGULAR' audit event.
    """
    # Remove records where DESC (audit event) != REGULAR
    df = df[df.DESC == 'REGULAR']

    return df
def add_hourly_entries(df):
    """
    The 'ENTRIES' variable recorded in the MTA data are cumulative entries of the turnstile per row. 
    Considering the data for a single turnstile machine (unique SCP, C/A, and UNIT),
    we want to add a new column symbolizing the incremental number of entries since the last recording time.
    
    This function is to add a new column, calculate the difference between ENTRIES in the current row and the
    previous row, and assign the difference to the new column. When there is NaN, fill it with 0.
    """
    
    HOURLY_ENTRIES = df.ENTRIES - df.ENTRIES.shift(1) 
    df['HOURLY_ENTRIES'] = HOURLY_ENTRIES.fillna(0)
    return df


def add_hourly_exits(df):
    """
    The 'EXITS' variable recorded in the MTA data are cumulative exits of the turnstile per row. 
    Considering the data for a single turnstile machine (unique SCP, C/A, and UNIT),
    we want to add a new column symbolizing the incremental number of exits since the last recording time.
    
    This function is to add a new column, calculate the difference between EXITS in the current row and the
    previous row, and assign the difference to the new column. When there is NaN, fill it with 0.
    """
    
    HOURLY_EXITS = df.EXITS - df.EXITS.shift(1) 
    df['HOURLY_EXITS'] = HOURLY_EXITS.fillna(0)
    return df

def add_busyness(df):
    """
    Define busyness as sum of ebtries and exits. Add a new column and assign busyness to it.
    """
    
    BUSYNESS = df.HOURLY_ENTRIES + df.HOURLY_EXITS 
    df['BUSYNESS'] = BUSYNESS
    return df


def time_to_hour(time):
    """
    Input 00:00:00 (hour:minute:second).
    Extract and return the hour from input.
    """
    # return pd.to_datetime(time).hour
    return int(time.split(':')[0])

def date_to_month(date):
    """
    Input mm-dd-yy (month-day-year).
    Extract and return the month from input date.
    """
    # return pd.to_datetime(date).month
    return int(date.split('/')[0])
               
def date_to_year(date):
    """
    Input sting mm-dd-yy (month-day-year).
    Extract and return the year from input date.
    """
    # return pd.to_datetime(date)
    return int(date.split('/')[2])


In [ ]:
def MTA_data(url,start,stop,log_name,date_tag):
    
    # download data
    df = download_data(url,start,stop,log_name)
    df.to_csv('MTA_data_{}.csv'.format(date_tag))
    
    # filter illegitimate data
    df = remove_irregular_event(df)
    df.to_csv('MTA_data_regular_{}.csv'.format(date_tag))
    
    # add hourly incremental entries
    df = df.groupby(['C/A','UNIT','SCP']).apply(
        add_hourly_entries)
    
    # add hourly incremental exits
    df = df.groupby(['C/A','UNIT','SCP']).apply(
        add_hourly_exits)

    # add a 'HOUR', 'MONTH' and 'YEAR' column 
    df['HOUR'] = df['TIME'].map(time_to_hour)
    df['MONTH'] = df['DATE'].map(date_to_month)
    df['YEAR'] = df['DATE'].map(date_to_year)
    
    df.to_csv('MTA_data_hour_{}.csv'.format(date_tag))
    
    return df

In [ ]:
url = "http://web.mta.info/developers/turnstile.html"
jpy = pq(url)
for i in range(2*len(jpy('#contentbox > div > div > a'))):
    if (jpy('#contentbox > div > div > a:nth-child({})'.format(i)).text() == "Saturday, October 18, 2014"):
        stop = i+1
        print("end_point = " + str(i))
    if (jpy('#contentbox > div > div > a:nth-child({})'.format(i)).text() == "Saturday, December 27, 2014"):
        start = i
        print("start_point = " + str(i))
        
log_name = 'Download_Log_new.txt'
date_tag = 'y2014_new' # change here accordingly
df = MTA_data(url,start,stop,log_name,date_tag)
print(df.shape)
df.head()

In [ ]:
df = pd.read_csv('./MTA_data_hour_y2014_new.csv')

In [ ]:
def clean_data(df):
    """
    HOURLY_ENTRIES and HOURLY_EXITS contains negative value and abnormally large value (ie. the max is 931476882).
    Assuming it takes 1 second for 1 people enter the turnstile, 
    there can be at max 14,400 people entering turnstile in 4 hours. 
    So in theory, considering a buffer, any HOURLY_ENTRIES (or HOURLY_EXITS) greater than 20000 is not possible. 
    Also HOURLY_ENTRIES and HOURLY_EXITS obviously cannot be negative.
    
    This function replace the negative and greater than 20000 HOURLY_ENTRIES by the mean of the group(ie.SCP,MONTH) that they are in.
    Then calculate the "BUSYNESS".
    """

    # clean 'HOURLY_ENTRIES'
    df['HOURLY_ENTRIES'] = df.groupby(['SCP','MONTH']).HOURLY_ENTRIES.transform(
        lambda x: np.where((x<0)|(x>20000),x.mask((x<0)|(x>20000)).mean(),x))
    
    # clean 'HOURLY_EXITS'
    df['HOURLY_EXITS'] = df.groupby(['SCP','MONTH']).HOURLY_EXITS.transform(
        lambda x: np.where((x<0)|(x>20000),x.mask((x<0)|(x>20000)).mean(),x))
    
    # add busyness
    df['BUSYNESS'] = df.HOURLY_ENTRIES + df.HOURLY_EXITS 
    
    return df

In [ ]:
df_final = clean_data(df)

In [ ]:
def hour_modified(hour):
    if hour > 20 or hour <= 0 : return 0
    if 0 < hour <= 4 : return 4
    if 4 < hour <= 8 : return 8
    if 8 < hour <= 12 : return 12
    if 12 < hour <= 16 : return 16
    if 16 < hour <= 20 : return 20

In [ ]:
mapping_final = pd.read_csv("./mapping_final.csv", sep=",")
df_master = pd.merge(df_final, mapping_final, how='left', on=['C/A','UNIT'], sort=True, copy=True, indicator=True)
df_master['Date_ID'] = (df_master['DATE']).astype('category').cat.codes 
df_master = df_master.rename(columns = {'Station ID':'Station_ID'})
df_master['HOUR_modified'] = df_master['HOUR'].map(hour_modified)

In [ ]:
df_master.to_csv(path_or_buf='./df_master_new.csv',sep=',', index=None)

In [ ]:
HOUR_LIST = [0,4,8,12,16,20]
col_name = ['Station_ID','HOURLY_ENTRIES','HOURLY_EXITS','Date','HOUR_modified','Station_name','Latitude','Longitude']
df_new = pd.DataFrame(columns = col_name)
for i in range(len(df_master['Date_ID'].unique())):
    
    for j in HOUR_LIST:
        temp = pd.DataFrame()
        temp = df_master[(df_master.Date_ID == i)&(df_master.HOUR_modified == j)].groupby('Station_ID')['HOURLY_ENTRIES','HOURLY_EXITS'].sum().round(2)
        temp = temp.reset_index()
        if len(temp)==0:
            continue
        else: 
            temp['Date'] = np.full((len(temp),1),df_master.loc[df_master.Date_ID == i,['DATE']].drop_duplicates().values)
            temp['HOUR_modified'] = np.full((len(temp),1),df_master.loc[df_master.HOUR_modified == j,['HOUR_modified']].drop_duplicates().values)
            temp = pd.merge(temp, df_master.loc[:,['Station_ID','Station_modified','GTFS Latitude','GTFS Longitude']].drop_duplicates(), how='left', on=['Station_ID']).rename(
            columns = {'Station_modified':'Station_name','GTFS Latitude':'Latitude','GTFS Longitude':'Longitude'})
            df_new = df_new.append(temp, ignore_index=True)

In [ ]:
df_new.to_csv(path_or_buf='./final_table_after.csv',sep=',',index=None)